
# Variable Definitions demo

We recommend using Vardef Models for creating and editing variable definitions. 

Useful models: 
- Draft
- Owner
- Contact
- UpdateDraft
- Patch
- ValidityPeriod

For alternatives read [Other input types](#alternatives-to-using-models-as-input)

In [1]:
from datetime import date
from pprint import pprint

from dapla_metadata.variable_definitions.generated.vardef_client.models.contact import (
    Contact,
)
from dapla_metadata.variable_definitions.generated.vardef_client.models.draft import (
    Draft,
)
from dapla_metadata.variable_definitions.generated.vardef_client.models.update_draft import (
    UpdateDraft,
)
from dapla_metadata.variable_definitions.generated.vardef_client.models.validity_period import (
    ValidityPeriod,
)
from dapla_metadata.variable_definitions.generated.vardef_client.models.variable_status import (
    VariableStatus,
)
from dapla_metadata.variable_definitions.vardef import Vardef



## Drafts

### Create a draft
Can produce a 409 CONFLICT if the `short_name` already exists.
Creator is saved as `Owner`.

All operations for created variable definition must be performed from a variable.

In [2]:
draft = Draft(
    name = {
        "nb": "test navn",
        "nn": "test namn",
        "en": "test name",
    },
    short_name= "test_shortname",
    definition= {
        "nb": "Skriv en definisjonstekst på norsk bokmål.",
        "nn": "Skriv ein definisjonstekst på nynorsk.",
        "en": "Write a definition text in english.",
    },
    classification_reference="91",
    unit_types=[
        "01",
    ],
    subject_fields=[
        "al",
    ],
    contains_special_categories_of_personal_data=False,
    measurement_type=None,
    valid_from=date(2024,12,18),
    external_reference_uri=None,
    comment=None,
    related_variable_definition_uris=None,
    contact=None,
)

# Assign value VariableDefinition to variable.
my_draft = Vardef.create_draft(draft)




### Example creating draft raising exception

Invalid Klass codes will trigger an exception. The client returns a VardefClientException with an exception message at the bottom (after the tracestack).

In [4]:
invalid_draft = Draft(
    name = {
        "nb": "test navn",
        "nn": "test namn",
        "en": "test name",
    },
    short_name= "test_excpetion",
    definition= {
        "nb": "Skriv en definisjonstekst på norsk bokmål.",
        "nn": "Skriv ein definosjonstekst på nynorsk.",
        "en": "Write a definition text in english.",
    },
    classification_reference="91",
    unit_types=[
        "haha",
    ],
    subject_fields=[
        "a",
    ],
    contains_special_categories_of_personal_data=False,
    measurement_type=None,
    valid_from=date(2024,12,18),
    external_reference_uri=None,
    comment=None,
    related_variable_definition_uris=None,
    contact=None,
)

Vardef.create_draft(invalid_draft)

VardefClientException: Status 400: 
createVariableDefinition.draft.unitTypes[0]<list element>: Code haha is not a member of classification with id 702
createVariableDefinition.draft.subjectFields[0]<list element>: Code a is not a member of classification with id 618


#### Useful information about variables (details)

If you need to assign your variable definition to a variable again and have access to your variable definition id:
> my_new_variable = Vardef.get_variable_definition("kWzf65gt") 

If you lost track of your variable definition id there are examples of how to fetch one particular variable definition in [Get one variable definition](#get-one-variable-definition)

In [22]:
# We can access all fields directly from the variable
print("Variable definition text in norwegian bokmål: ")
pprint(my_draft.definition["nb"])

# Or you can access fields by getting the saved Variable definition by id to check values
print("\nVariable definition created at: ")
print(Vardef.get_variable_definition(my_draft.id).created_at)

# Fields are not automatically updated after updates. If you wish to continue accessing fields directly from the variable you can update like this after every change:
my_draft = Vardef.get_variable_definition(my_draft.id)



Variable definition text in norwegian bokmål: 
'ny definisjon2'

Variable definition created at: 
2024-12-20 10:05:20.948000


### Edit a draft

This operation must be performed on Variable Definitions with status DRAFT. All operations on saved variable definitions are only available for `Variable owner`

In [5]:
# Update contact information
my_contact = Contact(
    title={
        "nb": "Seksjon for befolkningsstatistikk",
        "nn": "Seksjon for befolkningsstatistikk",
        "en": "Division for population statistics",
    },
    email="sibby@ssb.no",
)
update_contact = UpdateDraft(
    contact=my_contact,
)

my_draft.update_draft(update_draft=update_contact)
my_draft = Vardef.get_variable_definition(my_draft.id)

print(my_draft.contact)

{'title': {'nb': 'Seksjon for befolkningsstatistikk', 'nn': 'Seksjon for befolkningsstatistikk', 'en': 'Division for population statistics'}, 'email': 'sibby@ssb.no'}


### Example update draft raising exception
Invalid short name will trigger an `ValidationError`.

In [6]:
# Test draft exceptions invalid short_name
invalid_short_name = UpdateDraft(
    short_name="_)(45)",
)
my_draft.update_draft(invalid_short_name)


ValidationError: 1 validation error for UpdateDraft
short_name
  Value error, must validate the regular expression /^[a-z0-9_]{3,}$/ [type=value_error, input_value='_)(45)', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error

### Migrate a legacy definition from Vardok to Vardef

Note: This will raise an error if the short name already exists. After migration the variable is saved as `DRAFT`.



In [7]:
# Save variable Vardok
my_vardok_draft = Vardef.migrate_from_vardok("90")
print(my_vardok_draft)

{
  "id": "iHIE0tks",
  "patch_id": 1,
  "name": {
    "nb": "Spesifisert registreringstype",
    "nn": null,
    "en": null
  },
  "short_name": "spes_reg_type",
  "definition": {
    "nb": "Variabelen viser kode for personens særforhold i folkeregistreringen",
    "nn": null,
    "en": null
  },
  "classification_reference": null,
  "unit_types": [
    "20"
  ],
  "subject_fields": [],
  "contains_special_categories_of_personal_data": false,
  "variable_status": "DRAFT",
  "measurement_type": null,
  "valid_from": "1996-06-01",
  "valid_until": null,
  "external_reference_uri": "https://www.ssb.no/a/xml/metadata/conceptvariable/vardok/90",
  "comment": null,
  "related_variable_definition_uris": [],
  "owner": {
    "team": "dapla-felles",
    "groups": [
      "dapla-felles-developers"
    ]
  },
  "contact": null,
  "created_at": "2024-12-20T10:02:17.326655",
  "created_by": null,
  "last_updated_at": "2024-12-20T10:02:17.326655",
  "last_updated_by": null
}


#### Example migrate Vardok raising exception

In [8]:
# Id not found
Vardef.migrate_from_vardok("21")

VardefClientException: Status 404: Vardok id 21 not found

### Delete a draft

This operation can only be performed on Variable Definitions with status `DRAFT`.

In [9]:
my_vardok_draft.delete_draft()

'Variable iHIE0tks safely deleted'

## Publish

When draft has been quality checked it is time to publish the variable. 

Be aware: this process is irreversibel. 

In [10]:
update_status = UpdateDraft(
    variable_status=VariableStatus.PUBLISHED_INTERNAL,
)

print(my_draft.update_draft(update_status))

{
  "id": "d0iFCSjh",
  "patch_id": 1,
  "name": {
    "nb": "test navn",
    "nn": "test namn",
    "en": "test name"
  },
  "short_name": "test_shortname",
  "definition": {
    "nb": "Skriv en definisjonstekst på norsk bokmål.",
    "nn": "Skriv ein definisjonstekst på nynorsk.",
    "en": "Write a definition text in english."
  },
  "classification_reference": "91",
  "unit_types": [
    "01"
  ],
  "subject_fields": [
    "al"
  ],
  "contains_special_categories_of_personal_data": false,
  "variable_status": "PUBLISHED_INTERNAL",
  "measurement_type": null,
  "valid_from": "2024-12-18",
  "valid_until": null,
  "external_reference_uri": null,
  "comment": null,
  "related_variable_definition_uris": null,
  "owner": {
    "team": "dapla-felles",
    "groups": [
      "dapla-felles-developers"
    ]
  },
  "contact": {
    "title": {
      "nb": "Seksjon for befolkningsstatistikk",
      "nn": "Seksjon for befolkningsstatistikk",
      "en": "Division for population statistics"
    

## Patch

### Create a patch
Patches are to be used for minor changes which don't require a new Validity Period.

Examples of reasons for creating a new Patch:

- Correcting a typo
- Adding a translation
- Adding a subject field

Supply only the fields to be changed. Other fields will retain their current values.

#### Example update owner

Add owners

In [11]:
from dapla_metadata.variable_definitions.generated.vardef_client.models.owner import (
    Owner,
)
from dapla_metadata.variable_definitions.generated.vardef_client.models.patch import (
    Patch,
)

new_owner = Patch(
    owner=Owner(
    team="dapla-felles",
    groups=[
        "dapla-felles-developers",
        "play-enhjoern-a-developers",
    ],
),
)

my_draft.create_patch(new_owner)

my_draft = Vardef.get_variable_definition(my_draft.id)

print(my_draft.owner)

{'team': 'dapla-felles', 'groups': ['dapla-felles-developers', 'play-enhjoern-a-developers']}


#### Transfer ownership to another team
Be aware: when the team you represent is entirely remove as owner you will not have access to maintaining the variable definition.


In [12]:
# Create new variable definition for example ownership transferred
owner_draft = Draft(
    name = {
        "nb": "test navn",
        "nn": "test namn",
        "en": "test name",
    },
    short_name= "test_owner_2",
    definition= {
        "nb": "Skriv en definisjonstekst på norsk bokmål.",
        "nn": "Skriv ein definisjonstekst på nynorsk.",
        "en": "Write a definition text in english.",
    },
    classification_reference="91",
    unit_types=[
        "01",
    ],
    subject_fields=[
        "al",
    ],
    contains_special_categories_of_personal_data=False,
    measurement_type=None,
    valid_from=date(2024,12,18),
    external_reference_uri=None,
    comment=None,
    related_variable_definition_uris=None,
    contact=None,
)

transfer_owner_draft = Vardef.create_draft(owner_draft)

transfer_owner_draft.update_draft(UpdateDraft(variable_status=VariableStatus.PUBLISHED_INTERNAL))

replace_owner = Patch(
    owner=Owner(
    team="play-enhjoern-a",
    groups=[
        "play-enhjoern-a-developers",
    ],
),
)

transfer_owner_draft.create_patch(replace_owner)

CompletePatchOutput(id='weH43xV8', patch_id=2, name={'nb': 'test navn', 'nn': 'test namn', 'en': 'test name'}, short_name='test_owner_2', definition={'nb': 'Skriv en definisjonstekst på norsk bokmål.', 'nn': 'Skriv ein definisjonstekst på nynorsk.', 'en': 'Write a definition text in english.'}, classification_reference='91', unit_types=['01'], subject_fields=['al'], contains_special_categories_of_personal_data=False, variable_status=<VariableStatus.PUBLISHED_INTERNAL: 'PUBLISHED_INTERNAL'>, measurement_type=None, valid_from=datetime.date(2024, 12, 18), valid_until=None, external_reference_uri=None, comment=None, related_variable_definition_uris=None, owner={'team': 'play-enhjoern-a', 'groups': ['play-enhjoern-a-developers']}, contact=None, created_at=datetime.datetime(2024, 12, 20, 10, 3, 14, 327156), created_by=None, last_updated_at=datetime.datetime(2024, 12, 20, 10, 3, 14, 327156), last_updated_by=None)

#####  Attempt to create new patch after transferring ownership

In [13]:
transfer_owner_draft.create_patch(Patch(unit_types=["01"]))

VardefClientException: Status 403: Forbidden

### List patches for one variable definition

In [ ]:
patches = my_draft.list_patches()

print(patches)

### Get one patch

In [ ]:
print(my_draft.get_patch(2))

## Validity period

In order to create a new Validity Period input must contain updated 'definition' text for all present languages and a new valid from.

A new Validity Period should be created only when the fundamental definition of the variable has changed. 

This way the previous definition can be preserved for use in historical data.

In [14]:

valid_validity_period = ValidityPeriod(
    definition={
        "nb": "ny definisjon2",
        "nn": "ny definisjon2",
        "en": "new definition2",
    },
    valid_from=date(2040,4,5),
)
my_draft.create_validity_period(valid_validity_period)



CompletePatchOutput(id='d0iFCSjh', patch_id=4, name={'nb': 'test navn', 'nn': 'test namn', 'en': 'test name'}, short_name='test_shortname', definition={'nb': 'ny definisjon2', 'nn': 'ny definisjon2', 'en': 'new definition2'}, classification_reference='91', unit_types=['01'], subject_fields=['al'], contains_special_categories_of_personal_data=False, variable_status=<VariableStatus.PUBLISHED_INTERNAL: 'PUBLISHED_INTERNAL'>, measurement_type=None, valid_from=datetime.date(2040, 4, 5), valid_until=None, external_reference_uri=None, comment=None, related_variable_definition_uris=None, owner={'team': 'dapla-felles', 'groups': ['dapla-felles-developers', 'play-enhjoern-a-developers']}, contact={'title': {'nb': 'Seksjon for befolkningsstatistikk', 'nn': 'Seksjon for befolkningsstatistikk', 'en': 'Division for population statistics'}, 'email': 'sibby@ssb.no'}, created_at=datetime.datetime(2024, 12, 20, 10, 5, 20, 948595), created_by=None, last_updated_at=datetime.datetime(2024, 12, 20, 10, 5, 2

#### Example invalid validity period

In [15]:
invalid_validity_period = ValidityPeriod(
    name={
        "nb": "new name",
    },
    valid_from=date(2025,4,5),
)

my_draft.create_validity_period(invalid_validity_period)

ValidationError: 1 validation error for ValidityPeriod
definition
  Field required [type=missing, input_value={'name': {'nb': 'new name...tetime.date(2025, 4, 5)}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing

### List validity periods for one variable definition

In [16]:
validity_periods = my_draft.list_validity_periods()
print(validity_periods)


[CompletePatchOutput(id='d0iFCSjh', patch_id=3, name={'nb': 'test navn', 'nn': 'test namn', 'en': 'test name'}, short_name='test_shortname', definition={'nb': 'Skriv en definisjonstekst på norsk bokmål.', 'nn': 'Skriv ein definisjonstekst på nynorsk.', 'en': 'Write a definition text in english.'}, classification_reference='91', unit_types=['01'], subject_fields=['al'], contains_special_categories_of_personal_data=False, variable_status=<VariableStatus.PUBLISHED_INTERNAL: 'PUBLISHED_INTERNAL'>, measurement_type=None, valid_from=datetime.date(2024, 12, 18), valid_until=datetime.date(2040, 4, 4), external_reference_uri=None, comment=None, related_variable_definition_uris=None, owner={'team': 'dapla-felles', 'groups': ['dapla-felles-developers', 'play-enhjoern-a-developers']}, contact={'title': {'nb': 'Seksjon for befolkningsstatistikk', 'nn': 'Seksjon for befolkningsstatistikk', 'en': 'Division for population statistics'}, 'email': 'sibby@ssb.no'}, created_at=datetime.datetime(2024, 12, 2

## List all variable definitions

In [17]:
variable_definitions = Vardef.list_variable_definitions()
print(variable_definitions)


[VariableDefinition(id='d0iFCSjh', patch_id=4, name={'nb': 'test navn', 'nn': 'test namn', 'en': 'test name'}, short_name='test_shortname', definition={'nb': 'ny definisjon2', 'nn': 'ny definisjon2', 'en': 'new definition2'}, classification_reference='91', unit_types=['01'], subject_fields=['al'], contains_special_categories_of_personal_data=False, variable_status=<VariableStatus.PUBLISHED_INTERNAL: 'PUBLISHED_INTERNAL'>, measurement_type=None, valid_from=datetime.date(2040, 4, 5), valid_until=None, external_reference_uri=None, comment=None, related_variable_definition_uris=None, owner={'team': 'dapla-felles', 'groups': ['dapla-felles-developers', 'play-enhjoern-a-developers']}, contact={'title': {'nb': 'Seksjon for befolkningsstatistikk', 'nn': 'Seksjon for befolkningsstatistikk', 'en': 'Division for population statistics'}, 'email': 'sibby@ssb.no'}, created_at=datetime.datetime(2024, 12, 20, 10, 5, 20, 948000), created_by=None, last_updated_at=datetime.datetime(2024, 12, 20, 10, 5, 2

### Filter by date


In [18]:

desired_validity = date(2030,2,24)
filtered_variable_definitions = Vardef.list_variable_definitions(desired_validity)
print(filtered_variable_definitions)

[VariableDefinition(id='d0iFCSjh', patch_id=3, name={'nb': 'test navn', 'nn': 'test namn', 'en': 'test name'}, short_name='test_shortname', definition={'nb': 'Skriv en definisjonstekst på norsk bokmål.', 'nn': 'Skriv ein definisjonstekst på nynorsk.', 'en': 'Write a definition text in english.'}, classification_reference='91', unit_types=['01'], subject_fields=['al'], contains_special_categories_of_personal_data=False, variable_status=<VariableStatus.PUBLISHED_INTERNAL: 'PUBLISHED_INTERNAL'>, measurement_type=None, valid_from=datetime.date(2024, 12, 18), valid_until=datetime.date(2040, 4, 4), external_reference_uri=None, comment=None, related_variable_definition_uris=None, owner={'team': 'dapla-felles', 'groups': ['dapla-felles-developers', 'play-enhjoern-a-developers']}, contact={'title': {'nb': 'Seksjon for befolkningsstatistikk', 'nn': 'Seksjon for befolkningsstatistikk', 'en': 'Division for population statistics'}, 'email': 'sibby@ssb.no'}, created_at=datetime.datetime(2024, 12, 20

### Writing your own filter operations
The APi does not support filter operations directly, but it easy to write your own.

In [19]:
print("\nFilter by team: ")
my_team_variables = [variable for variable in variable_definitions if variable.owner["team"] == "dapla-felles"]
print(my_team_variables)

print("\nFilter by status draft: ")
draft_variables = [variable for variable in variable_definitions if variable.variable_status == VariableStatus.DRAFT]
print(draft_variables)

print("\nFilter by status published internal: ")
published_intern_variables = [variable for variable in variable_definitions if variable.variable_status == VariableStatus.PUBLISHED_INTERNAL]
print(published_intern_variables)



Filter by team: 
[VariableDefinition(id='d0iFCSjh', patch_id=4, name={'nb': 'test navn', 'nn': 'test namn', 'en': 'test name'}, short_name='test_shortname', definition={'nb': 'ny definisjon2', 'nn': 'ny definisjon2', 'en': 'new definition2'}, classification_reference='91', unit_types=['01'], subject_fields=['al'], contains_special_categories_of_personal_data=False, variable_status=<VariableStatus.PUBLISHED_INTERNAL: 'PUBLISHED_INTERNAL'>, measurement_type=None, valid_from=datetime.date(2040, 4, 5), valid_until=None, external_reference_uri=None, comment=None, related_variable_definition_uris=None, owner={'team': 'dapla-felles', 'groups': ['dapla-felles-developers', 'play-enhjoern-a-developers']}, contact={'title': {'nb': 'Seksjon for befolkningsstatistikk', 'nn': 'Seksjon for befolkningsstatistikk', 'en': 'Division for population statistics'}, 'email': 'sibby@ssb.no'}, created_at=datetime.datetime(2024, 12, 20, 10, 5, 20, 948000), created_by=None, last_updated_at=datetime.datetime(2024

#### Get one variable definition 
Get one variable by id. If you loose track of your variable definition id or


In [ ]:
print(Vardef.get_variable_definition(my_draft.id))

#### Get variable definition id by short name

In [ ]:

variable_by_short_name = [variable for variable in variable_definitions if variable.short_name == "test_shortname"]
print(variable_by_short_name[0].id)

## Alternatives to using Models

Valid dictionary

In [39]:
NEW_DRAFT = {
    "name": {"en": "Country Background", "nb": "Landbakgrunn", "nn": "Landbakgrunn"},
    "short_name": "new_short_name1",
    "definition": {
        "en": "Country background is the person's own, the mother's or possibly the father's country of birth. Persons without an immigrant background always have Norway as country background. In cases where the parents have different countries of birth the mother's country of birth is chosen. If neither the person nor the parents are born abroad, country background is chosen from the first person born abroad in the order mother's mother, mother's father, father's mother, father's father.",
        "nb": "For personer født i utlandet, er dette (med noen få unntak) eget fødeland. For personer født i Norge er det foreldrenes fødeland. I de tilfeller der foreldrene har ulikt fødeland, er det morens fødeland som blir valgt. Hvis ikke personen selv eller noen av foreldrene er utenlandsfødt, hentes landbakgrunn fra de første utenlandsfødte en treffer på i rekkefølgen mormor, morfar, farmor eller farfar.",
        "nn": "For personar fødd i utlandet, er dette (med nokre få unntak) eige fødeland. For personar fødd i Noreg er det fødelandet til foreldra. I dei tilfella der foreldra har ulikt fødeland, er det fødelandet til mora som blir valt. Viss ikkje personen sjølv eller nokon av foreldra er utenlandsfødt, blir henta landsbakgrunn frå dei første utenlandsfødte ein treffar på i rekkjefølgja mormor, morfar, farmor eller farfar.",
    },
    "classification_reference": "91",
    "unit_types": ["01", "02"],
    "subject_fields": ["he04"],
    "contains_special_categories_of_personal_data": True,
    "measurement_type": None,
    "valid_from": "2003-01-01",
    "external_reference_uri": "https://www.ssb.no/a/metadata/conceptvariable/vardok/1919/nb",
    "comment": {
        "nb": "Fra og med 1.1.2003 ble definisjon endret til også å trekke inn besteforeldrenes fødeland.",
        "nn": "Fra og med 1.1.2003 ble definisjon endret til også å trekke inn besteforeldrenes fødeland.",
        "en": "As of 1 January 2003, the definition was changed to also include the grandparents' country of birth.",
    },
    "related_variable_definition_uris": ["https://example.com/"],
    "contact": {
        "title": {
            "en": "Division for population statistics",
            "nb": "Seksjon for befolkningsstatistikk",
            "nn": "Seksjon for befolkningsstatistikk",
        },
        "email": "s320@ssb.no",
    },
}

